In [1]:
# get the code
import sys
import os
sys.path.append('../code')

# set the paths
home = '/Users/martinscience'
#  home = '/Users/mahtin'


somvar_path = os.path.join(home, "Dropbox/Icke/Work/somVar")
testdata = os.path.join(somvar_path, "testdata")
ebdata = os.path.join(somvar_path, "tooldata/EBdata")
pon_path = os.path.join(testdata, "PON")

static = os.path.join(home, "Dropbox/Icke/Work/static")

## create EBmatrix from different input
+ filterVar.mawk is the input fork

In [ ]:
from ebraw import tumor2matrix

EBconfig = {
    "temp_dir": os.path.join(ebdata, "temp"),
    "pon_path": pon_path,
    "mawk_path": "../shell",
    "genome_split": os.path.join(static, "genome/gatk/hg38/split"),
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "count_dict": {0:"alt+", 1:"alt-", 2:"depth+", 3:"depth-"},
    "use_cache": True
}


bam_file = f"{testdata}/bam/003_A.chr7.bam"
pileup_file = os.path.join(ebdata, "pileup/002_A-B.pileup.gz")
mut_file = os.path.join(ebdata, "table/002_A-B.csv")
pon_list = "PON_chr7.txt"
chrom = "chr7"

# df = bam2matrix(mut_file, pileup=pileup_file, chrom=chrom, pon_list=pon_list, EBconfig=EBconfig)
df = tumor2matrix(mut_file, bam=bam_file, chrom=chrom, pon_list=pon_list, EBconfig=EBconfig)
df

## matrix to AB

### get_count_df for matrix2EB

In [ ]:
from ebutils import get_count_df
row = df.iloc[2,:]
row
cdf = get_count_df(row['PON:ALT=Depth'], EBconfig["count_dict"])
cdf

In [ ]:
from ebconvert import matrix2AB
from ebcore import fit_bb
bb_params = fit_bb(cdf, 0.5)
bb_params

In [ ]:
matrix2AB(row, config=EBconfig)

### expand matrix2AB to df

In [ ]:
df['ABparams'] = df.apply(matrix2AB, config=EBconfig, axis=1)
df[:3]

## AB to EB

In [ ]:
# make a row with the same values as in EBold for comparison
row_dict = {
    "Chr": ["chr7"],
    "Start": 680651,
    "Tumor:Alt=Depth": "0-5=12-42",
    "ABparams": "0.20509|23.07746-0.22661|2.47235"
}

test_row = pd.DataFrame().from_dict(row_dict).iloc[0]
test_row
from ebconvert import AB2EBscore
AB2EBscore(test_row, config=EBconfig)

In [ ]:
df['EBscore'] = df.apply(AB2EBscore, config=EBconfig, axis=1)
df